In [25]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
import matplotlib.pyplot as plt
import joblib

In [27]:
# Load dataset
df = pd.read_csv("C:/Users/Victus/Desktop/Project Self Learn/data/fctms_final_dataset.csv")

In [28]:
# Create simplified target label
experience_map = {1: "Beginner", 2: "Intermediate", 3: "Advanced"}
df["Experience_Level_Label"] = df["Experience_Level"].map(experience_map)
df["Workout_Label"] = df["Experience_Level_Label"] + " " + df["Type_of_Exercise"]

In [29]:
# Encode categorical features
label_encoders = {}
categorical_cols = ["Gender", "Type_of_Exercise", "Age_Group", "Workout_Label"]
for col in categorical_cols:
    le = LabelEncoder()
    df[col + "_Encoded"] = le.fit_transform(df[col])
    label_encoders[col] = le

In [36]:
# Reduce the influence of Type_of_Exercise_Encoded
df["Type_of_Exercise_Weighted"] = df["Type_of_Exercise_Encoded"] * 0.2

# Define features and target
features = [
    "Age", "Gender_Encoded", "BMI", "Fat_Percentage",
    "Session_Duration_Minutes", "Workout_Frequency (days/week)",
    "Experience_Level", "Type_of_Exercise_Weighted", "Age_Group_Encoded"
]

X = df[features]
y = df["Workout_Label_Encoded"]

In [37]:
# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

# Train RandomForest model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

In [38]:
# Save artifacts
joblib.dump(model, "fctms_rf_model.pkl")
joblib.dump(label_encoders, "fctms_label_encoders.pkl")
joblib.dump(scaler, "fctms_scaler.pkl")

# Plot and save accuracy
plt.figure(figsize=(5, 4))
plt.bar(["RandomForest"], [accuracy])
plt.title("Final Model Accuracy (Simplified Labels)")
plt.ylabel("Accuracy")
plt.ylim(0, 1)
plt.tight_layout()
plt.savefig("model_accuracy_plot.png")
plt.close()

print("Model Accuracy:", accuracy)

Model Accuracy: 1.0
